In [ ]:
# Uncovering Stories from Singapore’s HDB Resale Market

In [ ]:
# Over the past decade, Singapore’s HDB resale market has undergone notable changes — from cooling measures and COVID-19 disruptions to the emergence of million-dollar flats. But while much attention is paid to price trends and housing supply, the story of the agent is often overlooked.

# Using open datasets from CEA (agent transactions) and HDB (resale flat prices), we explored:

# Where agents are most active

# What types of flats they focus on

# How activity and pricing evolved through the pandemic

# Let’s explore what the data reveals.

In [ ]:
import requests
import time
import pandas as pd
from io import StringIO
import plotly.graph_objects as go
import warnings
warnings.filterwarnings(action="ignore")


CEA_DATASET_ID = "d_ee7e46d3c57f7865790704632b0aef71"
RESALE_DATASET_ID='d_8b84c4ee58e3cfc0ece0d773c8ca6abc'
max_tries=3

def load_dataset_as_dataframe(DATASET_ID):
    INITIATE_URL = f"https://api-open.data.gov.sg/v1/public/api/datasets/{DATASET_ID}/initiate-download"
    POLL_URL = f"https://api-open.data.gov.sg/v1/public/api/datasets/{DATASET_ID}/poll-download"
    init_resp = requests.get(INITIATE_URL)
    init_resp.raise_for_status()

    for _ in range(max_tries):
        time.sleep(2)
        poll_resp = requests.get(POLL_URL)
        poll_resp.raise_for_status()
        download_url = poll_resp.json().get("data", {}).get("url")
        if download_url:
            break
    else:
        raise TimeoutError("Timed out waiting for dataset download URL.")

    csv_resp = requests.get(download_url)
    csv_resp.raise_for_status()
    df = pd.read_csv(StringIO(csv_resp.text))
    return df

cea_df = load_dataset_as_dataframe(CEA_DATASET_ID)
resale_df = load_dataset_as_dataframe(RESALE_DATASET_ID)


In [ ]:
cea_df['transaction_type'].value_counts()

,count
transaction_type,
WHOLE RENTAL,560785
RESALE,452690
ROOM RENTAL,114684
NEW SALE,53781
SUB-SALE,176


In [ ]:
cea_df = cea_df[cea_df['transaction_type'] == 'RESALE']


In [ ]:
cea_df['month'] = pd.to_datetime(cea_df['transaction_date'], format='%b-%Y')

/tmp/ipython-input-37-3964039917.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
cea_df

,salesperson_name,transaction_date,salesperson_reg_num,property_type,transaction_type,represented,town,district,general_location,year,month
0,SOH HUI HUA,2017-10-01,P000003I,HDB,RESALE,SELLER,YISHUN,-,-,2017,2017-10-01
1,SOH HUI HUA,2018-03-01,P000003I,HDB,RESALE,BUYER,KALLANG/WHAMPOA,-,-,2018,2018-03-01
2,SOH HUI HUA,2017-01-01,P000003I,HDB,RESALE,BUYER,YISHUN,-,-,2017,2017-01-01
3,SOH HUI HUA,2018-05-01,P000003I,HDB,RESALE,SELLER,CHOA CHU KANG,-,-,2018,2018-05-01
4,SOH HUI HUA,2017-11-01,P000003I,HDB,RESALE,SELLER,CHOA CHU KANG,-,-,2017,2017-11-01
...,...,...,...,...,...,...,...,...,...,...,...
1182105,-,2023-12-01,-,HDB,RESALE,BUYER,BUKIT MERAH,-,-,2023,2023-12-01
1182106,-,2023-12-01,-,HDB,RESALE,SELLER,WOODLANDS,-,-,2023,2023-12-01
1182107,-,2023-12-01,-,HDB,RESALE,BUYER,HOUGANG,-,-,2023,2023-12-01
1182108,-,2023-12-01,-,HDB,RESALE,BUYER,YISHUN,-,-,2023,2023-12-01


In [ ]:
cea_df.describe()

,transaction_date,year,month
count,452690,452690.000000,452690
mean,2021-11-27 05:51:17.904968192,2021.451660,2021-11-27 05:51:17.904968192
min,2017-01-01 00:00:00,2017.000000,2017-01-01 00:00:00
25%,2020-05-01 00:00:00,2020.000000,2020-05-01 00:00:00
50%,2022-02-01 00:00:00,2022.000000,2022-02-01 00:00:00
75%,2023-11-01 00:00:00,2023.000000,2023-11-01 00:00:00
max,2025-07-01 00:00:00,2025.000000,2025-07-01 00:00:00
std,NaN,2.308618,NaN


In [ ]:
resale_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,year
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,2017
1,2017-01-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,2017
2,2017-01-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,2017
3,2017-01-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,2017
4,2017-01-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...
211753,2025-04-01,YISHUN,EXECUTIVE,828,YISHUN ST 81,04 TO 06,146.0,Maisonette,1988,61 years 09 months,975000.0,2025
211754,2025-05-01,YISHUN,EXECUTIVE,828,YISHUN ST 81,04 TO 06,142.0,Apartment,1988,61 years 09 months,1000000.0,2025
211755,2025-07-01,YISHUN,EXECUTIVE,824,YISHUN ST 81,04 TO 06,146.0,Maisonette,1987,61 years 05 months,980000.0,2025
211756,2025-05-01,YISHUN,MULTI-GENERATION,632,YISHUN ST 61,04 TO 06,147.0,Multi Generation,1987,61 years 06 months,945000.0,2025


In [ ]:
cea_df['transaction_date'] = pd.to_datetime(cea_df['transaction_date'], format="%b-%Y")
cea_df['year'] = cea_df['transaction_date'].dt.year

/tmp/ipython-input-42-1602021383.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-42-1602021383.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# The Resilient Agent: Post-COVID Recovery
# Even as lockdowns froze much of the economy in 2020, resale activity recovered strongly in 2021 and beyond.
# Agent transactions — which dipped due to viewing restrictions and uncertainty — surged back as buyers returned and prices began to climb.



In [ ]:
cea_resale = cea_df[cea_df['transaction_type'] == 'RESALE']

agent_counts = cea_resale.groupby('year').size().reset_index(name='agent_transactions')

resale_avg = resale_df.groupby('year')['resale_price'].mean().reset_index(name='avg_resale_price')

merged = pd.merge(agent_counts, resale_avg, on='year')

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Bar(x=merged['year'], y=merged['agent_transactions'], name="Agent Transactions", marker_color='skyblue'))
fig.add_trace(go.Scatter(x=merged['year'], y=merged['avg_resale_price'], name="Avg Resale Price", mode='lines+markers', yaxis='y2'))
fig.update_layout(
    title=" Agent Recovery in the Resale Market (Post-COVID)",
    xaxis_title="Year",
    yaxis=dict(title="Agent Transactions"),
    yaxis2=dict(title="Average Resale Price (SGD)", overlaying='y', side='right'),
    template="plotly_white", height=500
)
fig.show()


In [ ]:
#  Hidden Battlegrounds: Agent Activity vs Affordability
# While towns like Bishan and Queenstown dominate headlines with million-dollar flats, we found surprising hotspots of agent activity in more affordable towns
# like Yishun and Woodlands & Sengkang.

# These areas show:

# High agent transaction volume
# Moderate resale prices
# High flat turnover

In [ ]:
# Agent activity by town
town_agents = cea_resale.groupby('town').size().reset_index(name='agent_deals')

# Average resale price by town
town_prices = resale_df.groupby('town')['resale_price'].mean().reset_index(name='avg_price')

# Volume by town
town_volume = resale_df.groupby('town').size().reset_index(name='flats_sold')

# Merge
merged_town = town_agents.merge(town_prices, on='town').merge(town_volume, on='town')

# Plot
import plotly.express as px
fig = px.scatter(
    merged_town, x='avg_price', y='agent_deals', size='flats_sold', color='town',
    title="Agent Activity vs Affordability",
    labels={'avg_price': 'Avg Resale Price (SGD)', 'agent_deals': 'Agent Transactions'},
    size_max=60
)
fig.show()


In [ ]:
# The Flat Type Focus: What’s Keeping Agents Busy?
# Agents tend to specialize based on demand patterns. The 4-room flat emerges as the clear leader in transaction volume — striking a balance between size and affordability for families.

# Meanwhile, 5-room and Executive flats, while pricier, attract fewer deals — likely due to a smaller buyer pool or lower availability.

In [ ]:
flat_group = resale_df.groupby('flat_type').size().reset_index(name='resale_transactions')
flat_price = resale_df.groupby('flat_type')['resale_price'].mean().reset_index(name='avg_price')
flat_merged = pd.merge(flat_group, flat_price, on='flat_type')

import plotly.express as px
fig = px.bar(flat_merged.sort_values(by='resale_transactions', ascending=False),
    x='flat_type', y='resale_transactions', color='avg_price',
    title="Flat Types with Most Resale Transactions",
    labels={'resale_transactions': 'Resale Transactions', 'avg_price': 'Avg Resale Price'},
    color_continuous_scale='Blues'
)
fig.show()